In [1]:
import pandas as pd
import os
import numpy as np
root_path = './'

# 1 邻接表创建

In [2]:
class ArcNode():
  def __init__(self, adj_vex, next_arc, kind, time, money):
    self.adj_vex = adj_vex
    self.next_arc = next_arc
    self.kind = kind
    self.time = time
    self.money = money

class VNode():
  def __init__(self, data):
    self.data = data
    self.first_arc = None

class ALGraph():
    """ 邻接表-图 """
    def __init__(self):
        self.vertices = np.array([], dtype=VNode)
        self.vex_num = 0
        self.arc_num = 0
 
    def locate_vex(self, vertex):
        for i in range(self.vex_num):
            if self.vertices[i].data == vertex:
                return i
        return -1
 
    def create_dg(self, vex_data: list, head_vex: list, tail_vex: list, kind: list, time: list, money:list):
        """
            create directed graph(构建有向图)
            时间复杂度: O( max{n, e} ), n表示顶点数量，e表示弧数量
            vex_data: 所有顶点
            head_vex: 所有弧头
            tail_vex: 所有弧尾
            weight: 所有弧的权值
        """
        self.vex_num = len(vex_data)
        self.arc_num = len(head_vex)
        for i in range(self.vex_num):
            self.vertices = np.append(self.vertices, [VNode(vex_data[i])])
        for i in range(self.arc_num):
            tail_vex_index = self.locate_vex(tail_vex[i])
            head_vex_index = self.locate_vex(head_vex[i])
            first_arc = self.vertices[head_vex_index].first_arc
            arc_node = ArcNode(self.vertices[tail_vex_index].data, first_arc, kind[i], time[i], money[i])
            self.vertices[head_vex_index].first_arc = arc_node

In [3]:
cities = pd.read_csv(os.path.join(root_path, 'cities.csv'), header=None)
cities = cities.reset_index()
cities.columns = ["city_id","country", "city", "latitude", "longitude"]
cities

,city_id,country,city,latitude,longitude
0,0,Afghanistan,Kabul,34.4667,69.18330
1,1,Albania,Tirane,41.3000,19.81670
2,2,Algeria,Algiers,36.7000,3.13333
3,3,American Samoa,Pago Pago,-14.2667,-170.71700
4,4,Andorra,Andorra la Vella,42.5167,1.53333
...,...,...,...,...,...
194,194,Venezuela,Caracas,10.5000,-66.91670
195,195,Vietnam,Hanoi,21.0833,105.91700
196,196,Yugoslavia,Belgrade,44.8333,20.61670
197,197,Zambia,Lusaka,-15.4667,28.26670


In [4]:
#cities_id = dict(zip(cities['city'], cities['city_id']))
cities_id = cities[["city", "city_id"]]
cities_id.head()

,city,city_id
0,Kabul,0
1,Tirane,1
2,Algiers,2
3,Pago Pago,3
4,Andorra la Vella,4


In [5]:
routes = pd.read_csv(os.path.join(root_path, 'routes.csv'), header=None)
routes.dropna(axis=1, how='any', inplace=True)
routes.columns = ['origin', 'destination', 'kind', 'time', 'money']
routes = pd.merge(routes, cities_id, left_on='origin', right_on='city').rename(columns={"city_id":"origin_id"})
routes = pd.merge(routes, cities_id, left_on='destination', right_on='city').rename(columns={"city_id":"destination_id"})
routes.drop(columns=['city_x', 'city_y'], inplace=True)
routes

,origin,destination,kind,time,money,origin_id,destination_id
0,Abu Dhabi,Canberra (Use Sydney),plane,24.00,1339.00,186,10
1,Amsterdam,Canberra (Use Sydney),plane,21.30,804.00,127,10
2,Asmara,Canberra (Use Sydney),plane,44.00,3284.00,60,10
3,Asuncion,Canberra (Use Sydney),plane,57.00,2129.00,141,10
4,Athens,Canberra (Use Sydney),plane,21.83,870.00,75,10
...,...,...,...,...,...,...,...
1970,Tegucigalpa,Managua,plane,4.00,292.00,84,131
1971,Stanley,Stanley,boat,0.00,0.00,63,63
1972,Stanley,Stanley,plane,0.00,0.00,63,63
1973,Tokyo,Saipan,plane,4.00,625.00,95,134


有向图

In [6]:
vex_data = list(cities['city'])
head_vex = list(routes['origin'])
tail_vex = list(routes['destination'])
kind = list(routes['kind'])
time = list(routes['time'])
money = list(routes['money'])

In [7]:
G = ALGraph()
G.create_dg(vex_data, head_vex, tail_vex, kind, time, money)

In [8]:
min = np.inf
p = G.vertices[84].first_arc
while p.next_arc is not None:
  print(p.adj_vex)
  p = p.next_arc
print(p.adj_vex)

Managua
Managua
Managua
San Salvador
San Salvador
Panama
Guatemala
Washington DC
Singapore
Pretoria (Use Johannesburg)
London
Lima
Canberra (Use Sydney)


最优路径分析

In [31]:
def getWeight(G, v0, v, kind):
  min = np.inf
  if v0 == v:
    return 0
  p = G.vertices[v0].first_arc
  while p is not None:
    if G.locate_vex(p.adj_vex) == v:
      if kind == 'time':
        if p.time < min:
          min = p.time
      elif kind == 'money':
        if p.money < min:
          min = p.money
    p = p.next_arc
  return min

def dijkstra(G, origin, kind):
  v0 = G.locate_vex(origin)
  final = list(np.zeros(len(cities_id)))
  path = list(np.zeros(len(cities_id)))
  D = list(np.zeros(len(cities_id)))
  for v in range(G.vex_num):
    final[v] = False
    D[v] = getWeight(G, v0, v, kind)
    path[v] = -1
  D[v0] = 0
  final[v0] = True

  for i in range(G.vex_num - 1):
    min = np.inf
    for w in range(G.vex_num):
      if (final[w] is not True) and (D[w] < min):
        v=w
        min=D[w]
    final[v] = True
    for w in range(G.vex_num):
      if (final[w] is not True) and ((min + getWeight(G, v, w, kind)) < D[w]):
        D[w] = min + getWeight(G, v, w, kind)
        path[w] = v
  return path

In [35]:
path = dijkstra(G, 'Beijing', 'money')

In [37]:
v = 187
best_path = [187]
while path[v] != -1:
  best_path.append(path[v])
  v = path[v]
best_path.append(39)
best_path.reverse()

for index in best_path:
  print(G.vertices[index].data)

Beijing
Hanoi
Vientiane
Bangkok
Singapore
London
